#### CSML1020 Course Project - New Plant Diseases Dectection
#### Authors (Group 3): Paul Doucet, Jerry Khidaroo

#### Initilization

In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import seaborn as sns
import os

In [15]:
import tensorflow as tf


In [16]:
# Use this to disable GPU
tf.config.set_visible_devices([], 'GPU')

In [17]:
from keras.applications.vgg16 import VGG16
import keras
from sklearn.model_selection import train_test_split
from keras import backend
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

#### Data Importing

In [18]:
# Image Data Folders
# dataDirTrain = '../NewPlantDiseasesDatasetSample/train'
# dataDirValidate = '../NewPlantDiseasesDatasetSample/valid'
# dataDirTest = '../NewPlantDiseasesDatasetSample/test'

#dataDirTrain = '../NewPlantDiseasesDataset/train'
#dataDirValidate = '../NewPlantDiseasesDataset/valid'
#dataDirTest = '../NewPlantDiseasesDataset/test'

# From Kaggle Site
dataDirTrain = '../input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train'
dataDirValidate = '../input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid'
dataDirTest = '../input/newplantdiseasessample/NewPlantDiseasesDatasetSample/test'



#### Load Train Image Data with Best Augmentation Filters

In [19]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

# batch_size = 128
batch_size = 48
training_set = train_datagen.flow_from_directory(dataDirTrain, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_set = valid_datagen.flow_from_directory(dataDirValidate, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

train_num = training_set.samples
valid_num = valid_set.samples

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


#### Perform Hyper-Parameter Tuning on Selected Model

In [20]:
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
import keras

def create_model_VGG16(activation='softmax', learning_rate=0.01):
    base_model=VGG16(include_top=False,input_shape=(224,224,3))
    # base_model=VGG16(include_top=False,input_shape=(112,112,3))
    base_model.trainable=False

    classifier=keras.models.Sequential()
    classifier.add(base_model)
    classifier.add(Flatten())
    #classifier.add(Dense(38,activation='softmax'))
    classifier.add(Dense(38,activation=activation))

    opt = Adam(lr=learning_rate)
    
    #classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return classifier

In [21]:
# Importing Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers

def create_model_alexnet(activation='softmax', learning_rate=0.01):
    # Initializing the CNN
    classifier = Sequential()

    # Convolution Step 1
    classifier.add(Convolution2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=(224, 224, 3), activation = 'relu'))

    # Max Pooling Step 1
    classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
    classifier.add(BatchNormalization())

    # Convolution Step 2
    classifier.add(Convolution2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'))

    # Max Pooling Step 2
    classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'))
    classifier.add(BatchNormalization())

    # Convolution Step 3
    classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
    classifier.add(BatchNormalization())

    # Convolution Step 4
    classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
    classifier.add(BatchNormalization())

    # Convolution Step 5
    classifier.add(Convolution2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'))

    # Max Pooling Step 3
    classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
    classifier.add(BatchNormalization())

    # Flattening Step
    classifier.add(Flatten())

    # Full Connection Step
    classifier.add(Dense(units = 4096, activation = 'relu'))
    classifier.add(Dropout(0.4))
    classifier.add(BatchNormalization())
    classifier.add(Dense(units = 4096, activation = 'relu'))
    classifier.add(Dropout(0.4))
    classifier.add(BatchNormalization())
    classifier.add(Dense(units = 1000, activation = 'relu'))
    classifier.add(Dropout(0.2))
    classifier.add(BatchNormalization())
    classifier.add(Dense(units = 38, activation = activation))

    classifier.load_weights('../input/bestweights9/best_weights_9.hdf5')

    # we chose to train the top 2 conv blocks, i.e. we will freeze
    # the first 8 layers and unfreeze the rest:
    for i, layer in enumerate(classifier.layers[:20]):
        layer.trainable = False

    # Compiling the Model
    classifier.compile(optimizer=optimizers.SGD(lr=learning_rate, momentum=0.9, decay=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
   
    return classifier

In [22]:
def create_model_D(activation='softmax', learning_rate=0.01):
    classifier = Sequential()
    classifier.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    classifier.add(MaxPooling2D((2, 2)))
    classifier.add(Dropout(0.2))
    classifier.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    classifier.add(MaxPooling2D((2, 2)))
    classifier.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    classifier.add(MaxPooling2D((2, 2)))
    classifier.add(Flatten())
    classifier.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    classifier.add(Dense(38,activation=activation))
    
    opt = Adam(lr=learning_rate)
    classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return classifier

In [10]:
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from keras import regularizers

# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model_VGG16, verbose = 0)

# Define the parameters to try out
params = {'activation': ["softmax"],
          'batch_size': [32, 48, 64], 
          'epochs': [3, 5, 10],
          'learning_rate': [0.01, 0.001, 0.0001]}

# Create a randomize search cv object passing in the parameters to try
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(3),
                                   verbose=2)

X_train, y_train = next(training_set)

random_search_results = random_search.fit(X_train, y_train)

# Print best score and parameters
print("Best Score: ", random_search_results.best_score_, "and Best Params: ", random_search_results.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.01, epochs=3, batch_size=32, activation=softmax .


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


58892288/58889256 [==============================] - 1s 0us/step
[CV]  learning_rate=0.01, epochs=3, batch_size=32, activation=softmax, total=  24.7s
[CV] learning_rate=0.01, epochs=3, batch_size=32, activation=softmax .


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.7s remaining:    0.0s


[CV]  learning_rate=0.01, epochs=3, batch_size=32, activation=softmax, total=  21.4s
[CV] learning_rate=0.01, epochs=3, batch_size=32, activation=softmax .
[CV]  learning_rate=0.01, epochs=3, batch_size=32, activation=softmax, total=  21.2s
[CV] learning_rate=0.01, epochs=10, batch_size=64, activation=softmax 
[CV]  learning_rate=0.01, epochs=10, batch_size=64, activation=softmax, total= 1.1min
[CV] learning_rate=0.01, epochs=10, batch_size=64, activation=softmax 
[CV]  learning_rate=0.01, epochs=10, batch_size=64, activation=softmax, total= 1.0min
[CV] learning_rate=0.01, epochs=10, batch_size=64, activation=softmax 
[CV]  learning_rate=0.01, epochs=10, batch_size=64, activation=softmax, total= 1.0min
[CV] learning_rate=0.001, epochs=10, batch_size=64, activation=softmax 
[CV]  learning_rate=0.001, epochs=10, batch_size=64, activation=softmax, total= 1.0min
[CV] learning_rate=0.001, epochs=10, batch_size=64, activation=softmax 
[CV]  learning_rate=0.001, epochs=10, batch_size=64, acti

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 23.5min finished


Best Score:  -3.841557343800863 and Best Params:  {'learning_rate': 0.0001, 'epochs': 3, 'batch_size': 32, 'activation': 'softmax'}


In [24]:
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from keras import regularizers

from keras.callbacks import ModelCheckpoint

# checkpoint for Alexnet Model
#weightpath = "../input/bestweights9/best_weights_9.hdf5"
#checkpoint = ModelCheckpoint(weightpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
#callbacks_list = [checkpoint]# checkpoint


# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model_alexnet, verbose = 0)

# Define the parameters to try out
params = {'activation': ["softmax"],
          'batch_size': [32, 48, 64], 
          'epochs': [3, 5, 10],
          'learning_rate': [0.01, 0.001, 0.0001]}

# Create a randomize search cv object passing in the parameters to try
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(3),
                                   verbose=1)

X_train, y_train = next(training_set)

random_search_results = random_search.fit(X_train, y_train)

# Print best score and parameters
print("Best Score: ", random_search_results.best_score_, "and Best Params: ", random_search_results.best_params_)

SyntaxError: invalid syntax (<ipython-input-24-f4fd04898dcb>, line 25)

In [ ]:
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from keras import regularizers

# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model_D, verbose = 0)

# Define the parameters to try out
params = {'activation': ["softmax"],
          'batch_size': [32, 48, 64], 
          'epochs': [3, 5, 10],
          'learning_rate': [0.01, 0.001, 0.0001]}

# Create a randomize search cv object passing in the parameters to try
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(3))

X_train, y_train = next(training_set)

random_search_results = random_search.fit(X_train, y_train)

# Print best score and parameters
print("Best Score: ", random_search_results.best_score_, "and Best Params: ", random_search_results.best_params_)

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from keras.optimizers import Adam
from keras import regularizers

# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model, verbose = 0)

# Define the parameters to try out
params = {'activation': ["softmax"],
          'batch_size': [32, 48, 64], 
          'epochs': [3, 5, 10],
          'learning_rate': [0.01, 0.001, 0.0001]}

# Create a grid search cv object passing in the parameters to try
random_search = GridSearchCV(model, params, cv = KFold(3))

X_train, y_train = next(training_set)

grid_search_results = random_search.fit(X_train, y_train)

# Print best score and parameters
print("Best Score: ", grid_search_results.best_score_, "and Best Params: ", grid_search_results.best_params_)

#### Inference